<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [7]:
# use requests.get() method with the provided static_url
# assign the response to a object

import requests

# Static URL from which we will scrape the data
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check the status code to ensure the request was successful
if response.status_code == 200:
    print("Request successful!")
else:
    print("Request failed. Status code:", response.status_code)

# Now 'response' contains the HTML content of the page





Request successful!


Create a `BeautifulSoup` object from the HTML `response`


In [ ]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute

from bs4 import BeautifulSoup

# Assume you've already fetched the page using requests
# and stored the response in a variable named 'response'

# Check if the request was successful
if response.status_code == 200:
    # Create a BeautifulSoup object for HTML parsing
    soup = BeautifulSoup(response.text, 'html.parser')
    print("BeautifulSoup object created successfully!")
else:
    print("Failed to fetch page, status code:", response.status_code)

# 'soup' now contains the parsed HTML document



BeautifulSoup object created successfully!


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [10]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`

# Assuming you have a BeautifulSoup object named 'soup' from the previous step

# Find all tables in the HTML content
html_tables = soup.find_all('table')

# Check how many tables were found
print("Number of tables found:", len(html_tables))

# Optional: Print out each table's 'class' attribute to help identify them
for i, table in enumerate(html_tables):
    print("Table", i, "class:", table.get('class'))



Number of tables found: 25
Table 0 class: ['col-begin']
Table 1 class: ['col-begin']
Table 2 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 3 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 4 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 5 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 6 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 7 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 8 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 9 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 10 class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 11 class: ['wikitable']
Table 12 class: ['wikitable']
Table 13 class: ['wikitable']
Table 14 class: ['wikitable']
Table 15 class: ['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
Table 16 class: ['nowraplinks', 'navbox-subgroup']
Table 17 class: ['nowraplinks', 'navbox-subgroup']
Table 18 class: ['nowraplinks', 'm

Starting from the third table is our target table contains the actual launch records.


In [11]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [14]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


# Assuming first_launch_table has been identified correctly as shown previously
# Find all th elements within the first row of the table header
header_row = first_launch_table.find('tr')  # Find the first row, which contains the headers
headers = header_row.find_all('th')

# List to store the column names extracted from the table headers
column_names = []

# Function provided earlier to clean and extract text from the header
def extract_column_from_header(row):
    if row.br:
        row.br.extract()  # Remove <br/> tags to prevent texts from sticking together
    if row.a:
        row.a.extract()  # Remove <a> tags to clean the text
    if row.sup:
        row.sup.extract()  # Remove <sup> tags to remove references and notes

    column_name = ' '.join(row.contents).strip()  # Join contents and strip whitespace

    if not(column_name.isdigit()):  # Ignore any headers that are digits (usually not headers)
        return column_name

# Iterate through each th element in the headers
for th in headers:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:  # Ensure the name is non-empty
        column_names.append(name)

# Check the extracted column names
print(column_names)


['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [15]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [17]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [22]:
extracted_row = 0
# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table row
    for rows in table.find_all("tr"):
        # Check to see if the first table heading is a number corresponding to a launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Get table elements
        row = rows.find_all('td')
        # If it is a number save cells in a dictionary
        if flag:
            extracted_row += 1
            # Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist = date_time(row[0])
            
            # Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
              
            # Append the booster version into launch_dict with key `Version Booster`
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else None
            launch_dict['Version Booster'].append(bv)
            
            # Append the launch site into launch_dict with key `Launch Site`
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch site'].append(launch_site)
            
            # Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)
            
            # Append the payload mass into launch_dict with key `Payload Mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)
            
            # Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)
            
            # Append the launch outcome into launch_dict with key `Launch Outcome`
            launch_outcome = row[7].get_text().strip()
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Append the booster landing status into launch_dict with key `Booster Landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)


In [19]:
print(launch_dict.keys())


dict_keys(['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time'])


In [24]:
import pandas as pd


df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})


print(df.head())


df.to_csv('spacex_web_scraped.csv', index=False)

print("Data exported to 'spacex_web_scraped.csv' successfully.")


  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
1          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
2          1       CCAFS  Dragon Spacecraft Qualification Unit            0   
3          1       CCAFS                                Dragon       525 kg   
4          2       CCAFS                                Dragon     4,700 kg   

  Orbit Customer Launch outcome Version Booster Booster landing  \
0   LEO   SpaceX        Success  F9 v1.0B0003.1         Failure   
1   LEO   SpaceX     Success[9]  F9 v1.0B0003.1         Failure   
2   LEO     NASA    Success[20]  F9 v1.0B0003.1    No attempt\n   
3   LEO     NASA        Success  F9 v1.0B0003.1      No attempt   
4   LEO     NASA        Success  F9 v1.0B0004.1    No attempt\n   

              Date   Time  
0      4 June 2010  18:45  
1      4 June 2010  18:45  
2      4 June 2010  18:45  
3      4 J

In [25]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [26]:
import requests
import pandas as pd

# Perform the GET request
url = "https://api.spacexdata.com/v4/launches"  
response = requests.get(url)
data = response.json()

# Convert the JSON response to a DataFrame
df = pd.json_normalize(data)

if 'static_fire_date_utc' in df.columns and pd.notna(df['static_fire_date_utc'][0]):
    # Extract the year from the 'static_fire_date_utc' column of the first row
    year = pd.to_datetime(df['static_fire_date_utc'][0]).year
else:
    year = "Date not available or column missing"

print("Year of static fire date in the first row:", year)


Year of static fire date in the first row: 2006


In [31]:
import requests
import pandas as pd

# Perform the GET request to fetch launches data
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
data = response.json()


df = pd.json_normalize(data)

if 'rocket' in df.columns:
    
    falcon_9_df = df[df['rocket'].str.contains("Falcon 9", na=False)]
    count_falcon_9 = len(falcon_9_df)
else:
    count_falcon_9 = "Rocket data column missing"

print("Number of Falcon 9 launches:", count_falcon_9)


Number of Falcon 9 launches: 0


In [30]:
import requests
import pandas as pd


url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
data = response.json()


print(data[:3])  # Adjust as necessary to view more or specific parts of the data


if data and isinstance(data[0], dict):
    keys = data[0].keys()
    print("Top-level keys:", keys)


if 'cores' in data[0]:
    print("Details of 'cores' from the first launch:", data[0]['cores'])
    if all('landingPad' in core for core in data[0]['cores'] if isinstance(core, dict)):
        print("Landing pad details are nested inside 'cores'")
        df = pd.json_normalize(data, record_path=['cores'], errors='ignore')
        missing_landingPad = df['landingPad'].isna().sum()
        print("Number of missing values in the 'landingPad' column:", missing_landingPad)
    else:
        print("Landing pad details are not consistently present or differently named")
else:
    print("Expected structure does not match; 'cores' key not found")



[{'fairings': {'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}, 'links': {'patch': {'small': 'https://images2.imgbox.com/94/f2/NN6Ph45r_o.png', 'large': 'https://images2.imgbox.com/5b/02/QcxHUb5V_o.png'}, 'reddit': {'campaign': None, 'launch': None, 'media': None, 'recovery': None}, 'flickr': {'small': [], 'original': []}, 'presskit': None, 'webcast': 'https://www.youtube.com/watch?v=0a_00nJ_Y88', 'youtube_id': '0a_00nJ_Y88', 'article': 'https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html', 'wikipedia': 'https://en.wikipedia.org/wiki/DemoSat'}, 'static_fire_date_utc': '2006-03-17T00:00:00.000Z', 'static_fire_date_unix': 1142553600, 'net': False, 'window': 0, 'rocket': '5e9d0d95eda69955f709d1eb', 'success': False, 'failures': [{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}], 'details': 'Engine failure at 33 seconds and loss of vehicle', 'crew': [], 'ships': [], 'capsules': [], 'payloads': ['5eb0e4b5b6c3bb0006eeb1e1

In [33]:
import pandas as pd


data = [
    {'fairings': {'reused': False}, 'links': {'patch': {'small': 'https://...'}}, 'cores': [{'core': '5e9e289df35918033d3b2623', 'landing_attempt': False, 'landpad': None}], 'flight_number': 1},
    {'fairings': {'reused': False}, 'links': {'patch': {'small': 'https://...'}}, 'cores': [{'core': '5e9e289ef35918416a3b2624', 'landing_attempt': False, 'landpad': None}], 'flight_number': 2},
    {'fairings': {'reused': False}, 'links': {'patch': {'small': 'https://...'}}, 'cores': [{'core': '5e9e289ef3591814873b2625', 'landing_attempt': False, 'landpad': None}], 'flight_number': 3}
]


df = pd.json_normalize(data, record_path='cores', meta=['flight_number'], errors='ignore')

missing_landpad_count = df['landpad'].isnull().sum()

print("Number of missing values in the 'landpad' column:", missing_landpad_count)



Number of missing values in the 'landpad' column: 3


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
